<a href="https://colab.research.google.com/github/nehalbaid10/Emotion-Recognizer/blob/main/training_em_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Dense, Flatten, Activation
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np


num_classes = 7
batch_size = 64
epochs = 100
Img_height = 48
Img_width = 48


data = pd.read_csv('/content/drive/MyDrive/emotion_2/fer2013(1).csv')
data.head()
  

In [ ]:
pixels = data['pixels'].tolist() 
faces = []

for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.asarray(face).reshape(Img_height, Img_width) 
    faces.append(face.astype('float32'))

faces = np.asarray(faces)
faces = np.expand_dims(faces, -1)

emotions = pd.get_dummies(data['emotion']).values


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(faces, emotions, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=41)




In [ ]:



emotions_model = Sequential()

# Block-1: The First Convolutional Block

emotions_model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', 
                 kernel_initializer='he_normal',
                 activation="relu", 
                 input_shape=(Img_height, Img_width, 1), 
                 name="Conv1"))

emotions_model.add(BatchNormalization(name="Batch_Norm1"))

emotions_model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', 
                 kernel_initializer='he_normal', 
                 activation="relu", name="Conv2"))

emotions_model.add(BatchNormalization(name="Batch_Norm2"))
model.add(MaxPooling2D(pool_size=(2,2), name="Maxemotions_pool1"))
model.add(Dropout(0.5, name="Dropout1"))emotions_

# Block-2: The Second Convolutional Block

emotions_model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', 
                 kernel_initializer='he_normal',
                 activation="relu", name="Conv3"))

emotions_model.add(BatchNormalization(name="Batch_Norm3"))

emotions_model.add(Conv2D(filters=64,kernel_size=(3,3),padding='same',
                 kernel_initializer='he_normal', 
                 activation="relu", name="Conv4"))

emotions_model.add(BatchNormalization(name="Batch_Norm4"))
model.add(MaxPooling2D(pool_size=(2,2), name="Maxemotions_pool2"))
model.add(Dropout(0.5, name="Dropout2"))emotions_

# Block-3: The Third Convolutional Block

emotions_model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', 
                 kernel_initializer='he_normal', 
                 activation="relu", name="Conv5"))

emotions_model.add(BatchNormalization(name="Batch_Norm5"))

emotions_model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', 
                 kernel_initializer='he_normal',
                 activation="relu", name="Conv6"))

emotions_model.add(BatchNormalization(name="Batch_Norm6"))
model.add(MaxPooling2D(pool_size=(2,2), name="Maxemotions_pool3"))
model.add(Dropout(0.5, name="Dropout3"))emotions_

# Block-4: The Fully Connected Block

emotions_model.add(Flatten(name="Flatten"))
model.add(Dense(64, activation="reemotions_lu", kernel_initializer='he_normal', name="Dense"))
model.add(BatchNormalization(name="Batch_Norm7"))emotions_
model.add(Dropout(0.5, name="Dropout4"))emotions_

# Block-5: The Output Block

emotions_model.add(Dense(num_classes, activation="softmax", kernel_initializer='he_normal', name = "Output"))

In [ ]:
from tensorflow import keras
from keras.utils.vis_utils import plot_model
from keras.utils import np_utils

keras.utils.plot_model(model, to_file='model3.png', show_layer_names=True)




model.compile(loss="categorical_crossentropy",
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])



from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping

tensorboard = TensorBoard(log_dir='logs3')
lr_reducer = ReduceLROnPlateau(monitor='val_accuracy', factor=0.9, patience=3, verbose=1)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=8, verbose=1, mode='auto')
checkpointer = ModelCheckpoint("drive/MyDrive/emotion_2/saved_model/emotions_part2.h5", monitor='val_accuracy', verbose=1, save_best_only=True)



model.fit(np.array(X_train), np.array(y_train),
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(np.array(X_test), np.array(y_test)),
          shuffle=True,
          callbacks=[lr_reducer, tensorboard, early_stopper, checkpointer])




model.save('drive/MyDrive/emotion_2/saved_model/em_part2.h5')
